In [1]:
import pandas as pd
import numpy as np
import gc
import catboost as cat
from catboost import CatBoostClassifier
import os 
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


In [2]:
train_root_path ='./data/train_preliminary/'
train_ad_path = os.path.join(train_root_path,'ad.csv')
train_click_path = os.path.join(train_root_path,'click_log.csv')
train_user_path = os.path.join(train_root_path,'user.csv')

In [3]:
test_root_path = './data/test/'
test_ad_path = os.path.join(test_root_path,'ad.csv')
test_click_path = os.path.join(test_root_path,'click_log.csv')

In [4]:
df_user = pd.read_csv(train_user_path)

In [6]:
aggcol = ['creative_id','ad_id','product_id','product_category','advertiser_id','industry','time','click_times']

In [7]:
tf_dfs =[]
for col in aggcol:
    f1 = pd.read_csv(f'./model/cache/{col}_tfidf.csv')
    tf_dfs.append(f1)
tf_feature = pd.concat(tf_dfs, axis=1, sort=False)

In [8]:
user_embed = pd.read_parquet(f'./model/cache/user_embed_creative_id.parquet')

In [12]:
tf_feature = pd.concat([tf_feature,user_embed], axis=1, sort=False)

In [16]:
tf_feature[['user_emb_'+str(i) for i in range(128)]] = pd.DataFrame(tf_feature.creative_id_user_embed.tolist(), index= tf_feature.index)


In [21]:
tf_final = tf_feature.drop('creative_id_user_embed',axis=1)

In [23]:
tf_final.shape

(1900000, 528)

In [ ]:
tf_feature.shape

In [24]:
train_feature = tf_final[:len(df_user)]
train_feature['age'] = df_user['age'] - 1
train_feature['gender'] = df_user['gender'] - 1

/Users/martindu/opt/anaconda3/envs/tfenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/martindu/opt/anaconda3/envs/tfenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
test_feature = tf_final[len(df_user):]

In [27]:
cols = [col for col in train_feature.columns if col not in ['age','gender']]

In [ ]:
cols

In [40]:
def evaluate_5_fold(train_df, test_df, cols,label_col,numclass, test=False):
    kf = KFold(n_splits=5, shuffle=True, random_state=1017)
    y_test = 0
    oof_train = np.zeros((train_df.shape[0], numclass))
    for i, (train_index, val_index) in enumerate(kf.split(train_df[cols],train_df[label_col])):
        X_train, y_train = train_df.loc[train_index, cols], train_df[label_col].values[train_index]
        X_val, y_val = train_df.loc[val_index, cols], train_df[label_col].values[val_index]

        
#         lgb_train = lgb.Dataset(
#             X_train, y_train)
#         lgb_eval = lgb.Dataset(
#             X_val, y_val,
#             reference=lgb_train)
#         params = {
#             'boosting_type': 'gbdt',
#             'learning_rate' : 0.04, 
#             'verbose': 0,
#             'num_leaves':256,
#             'min_child_weight':30,
#             "num_class":6,
#             'objective':'multiclass', 
#             'feature_fraction':0.4,
#             'bagging_fraction':0.7, # 0.9是目前最优的
#             'bagging_freq':5,  # 3是目前最优的
#             'seed': 1017,
#             'metric':'multi_error',
#             'nthread': 50,
#         }
        
        model = CatBoostClassifier( iterations=100,
                                   learning_rate=1,
                                   random_strength=0.1,
                                   depth=8,loss_function='MultiClass',eval_metric='Accuracy')
#         model = CatBoostClassifier(iterations=100, depth=5,cat_features=categorical_features_indices,learning_rate=0.5, loss_function='Logloss',
#                             logging_level='Verbose')
        model.fit(X_train,y_train,eval_set=(X_val, y_val))#plot=True
        y_pred = model.predict_proba(X_val)
        if test:
            y_test += model.predict_proba(test_df[cols])
        oof_train[val_index] = y_pred
    auc = accuracy_score(train_df[label_col].values, np.argmax(oof_train, axis=1))
    y_test /= 5
    #feature_list = pd.DataFrame()
    #feature_list['names'] = cols
    #feature_list['imp'] = model.get_feature_importance()
    #feature_list = feature_list.sort_values(by='imp', ascending=False)
    #print(feature_list)
    print('5 Fold auc:', auc)
    gc.collect()
    return auc, oof_train, y_test#, feature_list

In [ ]:
auc, oof_train, y_test = evaluate_5_fold(train_feature, test_feature, cols,'age',10, True)

In [31]:
np.save('./model/cache/age_train.predict.npy',oof_train) 
np.save('./model/cache/age_test.predict.npy',y_test) 

In [ ]:
auc_gender, oof_train_gender, y_test_gender = evaluate_5_fold(train_feature, test_feature, cols,'gender', 2 , True)

In [42]:
np.save('./model/cache/gender_train.predict.npy',oof_train_gender) 
np.save('./model/cache/gender_test.predict.npy',y_test_gender) 

In [43]:
y_test_gender

array([[0.99305396, 0.00694604],
       [0.00355018, 0.99644982],
       [0.34195761, 0.65804239],
       ...,
       [0.98959732, 0.01040268],
       [0.98725318, 0.01274682],
       [0.99624617, 0.00375383]])

In [44]:
age_arg = np.argmax(y_test,axis=1)+1
gender_arg = np.argmax(y_test_gender,axis=1)+1

In [49]:
df_data =pd.read_parquet('./data/cache/df_data.parquet')

In [51]:
df_test_final = df_data[df_data.age.isna()]

In [52]:
final = pd.DataFrame({'user_id':df_test_final['user_id'],'predicted_age':age_arg,'predicted_gender':gender_arg})
final.to_csv('./result/tree_0606.csv',index=False)

In [53]:
gc.collect()

4

In [55]:
tfidf = tf_feature.drop(['user_emb_'+str(i) for i in range(128)],axis=1)

In [57]:
tfidf = tfidf.drop('creative_id_user_embed',axis=1)

In [59]:
tfidf.to_parquet('./data/cache/df_tfidf_6model_result.parquet',index=False)